In [ ]:
import pandas as pd
import spacy
import string
import spacy.cli

In [ ]:
nlp = spacy.load('en_core_web_lg')

In [ ]:
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
sw_nltk = stopwords.words('english')

In [ ]:
data = pd.read_csv('../input/amazon-ml-challenge-2021-hackerearth/train.csv',escapechar='\\',quoting = 3)

In [ ]:
data = data.head(1000000)

In [ ]:
data.dropna(how = 'all',subset = ['TITLE','DESCRIPTION','BULLET_POINTS','BRAND'],inplace = True)
data.dropna(how = 'all',subset = ['TITLE','DESCRIPTION','BULLET_POINTS'],inplace = True)
data['BRAND'].replace(np.nan,'unbranded',inplace = True)
data.replace(np.nan,'',inplace = True)

In [ ]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [ ]:
def cleaner(text):
    if text == '':
        return ''
    text = remove_html_tags(text)
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = [word for word in text.split(' ') if word not in sw_nltk]
    return " ".join(words)

In [ ]:
data['TITLE'] = data['TITLE'].apply(lambda x : cleaner(x))
data['DESCRIPTION'] = data['DESCRIPTION'].apply(lambda x : cleaner(x))
data['BULLET_POINTS'] = data['BULLET_POINTS'].apply(lambda x : cleaner(x))
data['BRAND'] = data['BRAND'].apply(lambda x : cleaner(x))

In [ ]:
data.to_pickle('./partiallyCleanedData_K.pkl')

In [ ]:
data = pd.read_pickle("./partiallyCleanedData_K.pkl")

In [ ]:
data.head()

In [ ]:
! pip install git+https://github.com/LIAAD/yake
import yake

In [ ]:
keyword_extractor=yake.KeywordExtractor(top = 10,n= 2)

In [ ]:
def extract_keywords(text):
    keywords = keyword_extractor.extract_keywords(text)
    return ' '.join([kw[0] for kw in keywords])

In [ ]:
for index, row in data.iterrows():
    if data.at[index,'BULLET_POINTS'] != '':
        continue
    if data.at[index,'BULLET_POINTS'] == '' and data.at[index,'DESCRIPTION'] !='' :
        data.at[index,'BULLET_POINTS'] = extract_keywords(data.at[index,'DESCRIPTION'])
    elif data.at[index,'BULLET_POINTS'] == '' and data.at[index,'DESCRIPTION'] =='' and data.at[index,'TITLE'] !='':
        data.at[index,'BULLET_POINTS'] = extract_keywords(data.at[index,'TITLE'])

In [ ]:
data.head()

In [ ]:
finalDF = data[data.columns[2:]]

In [ ]:
finalDF.to_pickle('./finalData_K.pkl')

In [ ]:
finalDF.to_csv('./finalData_K.csv')